## Machine learning models with variable importance

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor

### 1. Preprocessing
1. Preprocess NYT data: read in, parse to most recent day, and calculate cumulative death rate and store result in column `death_rate`
2. Preprocess PM data: remove irrelevant columns, rename columns, modify datatypes of certain columns.
3. Zip together the NYT and PM data by the FIP code and the date. Split into training and test sets based on 85% training and 15% testing.

**Preprocess NYT data**

Here we calculate the cumulative death rate and store in the column `death_rate`. We also modify the `date` column to a datetime object.

In [2]:
# Read in NYT data
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
nyt_df = pd.read_csv(url,index_col=None,parse_dates=[0])

# Calculate cumulative death rate
nyt_df['death_rate'] = nyt_df['deaths'] / nyt_df['cases']

# Convert date column to datetime object
nyt_df['date'] = pd.to_datetime(nyt_df['date'], format='%d/%m/%Y')

**Preprocess PM data**

Here, we drop the unnecessary columns `Unnamed: 0`, `Country_Region` (US for all entries), `Last_Update` (encoded in the `date` column), the `Province_State`, `Combined_Key`, and `Admin2` columns (all of which are summarized in the `fips` column), `year.x` (2016 for all entries), `year.y` (2012 for all entries), `older_Population` (encoded in `older_pecent`), `state`, `hash`, `dateChecked`, `Abbrev`, and `Recovered` (since we only have zero values listed in the DataFrame. We're also dropping non-cumulative data like `Confirmed`, `Deaths`, `Recovered`, `Active`, and columns with all NaNs like `beds`, `pending`, `hospitalizedCurrently`, `hospitalizedCumulative`, `inIcuCurrently`, `inIcuCumulative`, `onVentilatorCurrently`, `onVentilatorCumulative`, `recovered`, `death`, `hospitalized`

We also rename the `Long_` column to `Long`, `older_pecent` to `older_percent`, and `Deaths` to `county_deaths`.

Finally, we modify the `date` column to change the datatype from an int to a TimeStamp object.

In [3]:
# Read in PM data
data_big = pd.read_csv('../data/processed_data_05-03-2020.csv')

In [4]:
# Drop relevant columns
to_drop = ['Unnamed: 0', 'Country_Region', 'Last_Update', 'Province_State', 'Admin2', 'year.x', 
           'year.y', 'older_Population', 'state', 'hash', 'dateChecked', 'Abbrev', 'Recovered', 'Confirmed',
           'Deaths', 'Active', 'beds', 'pending', 'hospitalizedCurrently', 'hospitalizedCumulative', 
           'inIcuCurrently', 'inIcuCumulative', 'onVentilatorCurrently', 'onVentilatorCumulative', 'recovered',
           'death', 'hospitalized']
data_big = data_big.drop(to_drop, axis=1)

# Rename relevant columns
to_rename = {'Long_': 'Long', 'older_pecent': 'older_percent', 'Deaths': 'county_deaths'}
data_big = data_big.rename(columns=to_rename)

# Change date column to a TimeStamp day
data_big['date'] = pd.to_datetime(data_big['date'], format='%Y%m%d')

In [5]:
data_big.shape

(3086, 40)

**Zip together NYT and PM data**

Here we append the `cases`, `deaths`, and `death_rate` columns from the NYT dataset (encoded in `nyt_df`) to the PM data set (encoded in `data_big`) based on the FIPS and the date of measure. We store this information in `data_big` as `cum_cases`, `cum_deaths`, and `death_rate`, respectively.

We also renumber the `fips` column to start at $0$ and incriment by $1$, storing the key in the dictionaries `fip2idx` and `idx2fip`. We do the same for the `date` column.

Finally, we split the dataframe into train and test sets.

In [6]:
# Add cases, deaths, and death_rate columns from nyt_df to data_big
data_big = data_big.merge(nyt_df[['fips', 'date', 'cases', 'deaths', 'death_rate']], 
                          left_on=['fips', 'date'], right_on=['fips', 'date'])

# Rename relevant columns
to_rename2 = {'cases': 'cum_cases', 'deaths': 'cum_deaths'}
data_big = data_big.rename(columns=to_rename2)

In [7]:
# Create hashtables for fips fip2idx and idx2fip
fips = sorted(set(data_big.fips.values))
fip2idx = {fips[i]: i for i in range(len(fips))}
idx2fip = {i: fips[i] for i in range(len(fips))}

# Replace fips using hashtable
data_big['fips'] = data_big['fips'].map(fip2idx)

In [8]:
# Drop NaN rows and create predictors matrix
data_big = data_big.dropna()
covid_df = data_big[['mean_pm25', 'mean_winter_temp', 'mean_summer_temp', 'Lat', 'Long',
       'poverty', 'popdensity', 'medianhousevalue', 'pct_blk',
       'medhouseholdincome', 'pct_owner_occ', 'hispanic', 'education',
       'population', 'pct_asian', 'pct_native', 'pct_white', 'q_popdensity',
       'smoke_rate', 'mean_bmi', 'Population', 'older_percent', 'population_frac_county','cum_cases',
       'cum_deaths', 'death_rate']]

In [9]:
# Create predictors matrix X and response matricies y_cases and y_death_rate
X = covid_df.drop(['cum_cases', 'cum_deaths', 'death_rate'], axis=1)
y_cases = covid_df['cum_cases']
y_death_rate = covid_df['death_rate']

### Create AdaBoost model and quantify variable importance

In [10]:
# Instantiate, fit, report score
ab_regr_cases = AdaBoostRegressor(n_estimators=200)
ab_regr_cases.fit(X, y_cases)
ab_regr_cases.score(X, y_cases)

0.4542581061913795

In [12]:
# Determine variable importance
feature_impts = pd.DataFrame()
feature_impts['feature'] = X.columns
feature_impts['ab_cases'] = ab_regr_cases.feature_importances_

In [24]:
# Repeat for death rate
# Instantiate, fit, report score
ab_regr_death_rate = AdaBoostRegressor(n_estimators=600)
ab_regr_death_rate.fit(X, y_death_rate)
ab_regr_death_rate.score(X, y_death_rate)

0.04367144742396589

In [25]:
# Determine variable importance
feature_impts['ab_death_rate'] = ab_regr_death_rate.feature_importances_

In [26]:
feature_impts

,feature,ab_cases,ab_death_rate
0,mean_pm25,0.034899,0.016647
1,mean_winter_temp,0.029080,0.042690
2,mean_summer_temp,0.009801,0.093814
3,Lat,0.016766,0.035793
4,Long,0.052447,0.337721
5,poverty,0.043893,0.017757
6,popdensity,0.078332,0.003525
7,medianhousevalue,0.018515,0.003080
8,pct_blk,0.004152,0.031906
9,medhouseholdincome,0.050055,0.000000


### Create random forest model and quantify variable importance

In [30]:
# Instantiate, fit, report score
rf_regr_cases = RandomForestRegressor()
rf_regr_cases.fit(X, y_cases)
rf_regr_cases.score(X, y_cases)

/Users/Josh/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9278111676584077

In [31]:
# Determine variable importance
feature_impts['rf_cases'] = rf_regr_cases.feature_importances_

In [32]:
# Repeat for death rate
# Instantiate, fit, report score
rf_regr_death_rate = RandomForestRegressor()
rf_regr_death_rate.fit(X, y_death_rate)
rf_regr_death_rate.score(X, y_death_rate)

/Users/Josh/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8038118010249118

In [34]:
# Determine variable importance
feature_impts['rf_death_rate'] = rf_regr_death_rate.feature_importances_
feature_impts

,feature,ab_cases,ab_death_rate,rf_cases,rf_death_rate
0,mean_pm25,0.034899,0.016647,0.001754,0.043609
1,mean_winter_temp,0.029080,0.042690,0.048664,0.090056
2,mean_summer_temp,0.009801,0.093814,0.025732,0.071084
3,Lat,0.016766,0.035793,0.012776,0.027834
4,Long,0.052447,0.337721,0.145042,0.109562
5,poverty,0.043893,0.017757,0.063269,0.029195
6,popdensity,0.078332,0.003525,0.179128,0.057867
7,medianhousevalue,0.018515,0.003080,0.002042,0.026348
8,pct_blk,0.004152,0.031906,0.005731,0.028305
9,medhouseholdincome,0.050055,0.000000,0.015910,0.025897


### Create gradient boosting model and quantify variable importance

In [35]:
# Instantiate, fit, report score
gb_regr_cases = GradientBoostingRegressor()
gb_regr_cases.fit(X, y_cases)
gb_regr_cases.score(X, y_cases)

0.9890835942052333

In [36]:
# Determine variable importance
feature_impts['gb_cases'] = gb_regr_cases.feature_importances_

In [37]:
# Repeat for death rate
# Instantiate, fit, report score
gb_regr_death_rate = GradientBoostingRegressor()
gb_regr_death_rate.fit(X, y_death_rate)
gb_regr_death_rate.score(X, y_death_rate)

0.5122396277760257

In [38]:
# Determine variable importance
feature_impts['gb_death_rate'] = gb_regr_death_rate.feature_importances_
feature_impts

,feature,ab_cases,ab_death_rate,rf_cases,rf_death_rate,gb_cases,gb_death_rate
0,mean_pm25,0.034899,0.016647,0.001754,0.043609,0.009412,0.031095
1,mean_winter_temp,0.029080,0.042690,0.048664,0.090056,0.031738,0.061631
2,mean_summer_temp,0.009801,0.093814,0.025732,0.071084,0.000479,0.058109
3,Lat,0.016766,0.035793,0.012776,0.027834,0.007006,0.021000
4,Long,0.052447,0.337721,0.145042,0.109562,0.173907,0.087836
5,poverty,0.043893,0.017757,0.063269,0.029195,0.056527,0.026317
6,popdensity,0.078332,0.003525,0.179128,0.057867,0.245304,0.018993
7,medianhousevalue,0.018515,0.003080,0.002042,0.026348,0.001303,0.011591
8,pct_blk,0.004152,0.031906,0.005731,0.028305,0.001392,0.023860
9,medhouseholdincome,0.050055,0.000000,0.015910,0.025897,0.010594,0.009207
